<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BANKAAI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. IMPORTS & API SETUP
import google.genai as genai
from google.genai import types
import os

# --- Secure API Client Initialization using Userdata ---
GEMINI_API_KEY = None
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI') # Or your specific secret name
except (ImportError, KeyError):
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

REQUESTED_MODEL_ID = 'gemini-3-pro-preview'
client = genai.Client(api_key=GEMINI_API_KEY) if GEMINI_API_KEY else None

if client:
    print(f"✅ Gemini client configured for **{REQUESTED_MODEL_ID}**.")

# --- 2. AGENTIC ORCHESTRATION FUNCTIONS ---

def agent_loan_processor(application_raw):
    """AGENT 1: The Processor (Low Thinking for Speed/Extraction)"""
    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_level="low"),
        temperature=1.0
    )
    prompt = f"Extract key financials: Loan Amount, Credit Score, and Income from this text: {application_raw}"

    response = client.models.generate_content(
        model=REQUESTED_MODEL_ID, config=config, contents=prompt
    )
    return response.text

def agent_senior_underwriter(processed_data):
    """AGENT 2: Senior Underwriter (High Thinking for Decision Logic)"""
    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_level="high",
            include_thoughts=True # This enables the 'Agentic reasoning' audit
        ),
        temperature=1.0
    )
    prompt = f"""
    Based on this processed loan data: {processed_data}

    DECISION LOGIC:
    - Approve if Credit > 700 AND Amount < $50k.
    - FLAG for Manual Review if Amount > $50k.
    - Reject if Credit < 600.

    Output a JSON with: 'decision', 'reasoning', 'risk_score'.
    """

    response = client.models.generate_content(
        model=REQUESTED_MODEL_ID, config=config, contents=prompt
    )
    return response

# --- 3. RUNNING THE ORCHESTRATOR ---

def run_banking_squad(user_query):
    print("🚀 Starting Agentic Handoff...")

    # Step 1: Processor Agent
    print("Step 1: Processor Agent (Thinking: Low) is extracting data...")
    data_summary = agent_loan_processor(user_query)

    # Step 2: Underwriter Agent
    print("Step 2: Underwriter Agent (Thinking: High) is deciding...")
    final_response = agent_senior_underwriter(data_summary)

    # Show the "Brain Trace" (Internal reasoning)
    if hasattr(final_response, 'thoughts'):
        print("\n🧠 INTERNAL AGENT REASONING (Audit Log):")
        print(final_response.thoughts)

    print("\n📄 FINAL BANKING DECISION:")
    print(final_response.text)

# --- EXECUTION ---
raw_app = "Customer: Alice. Wants to borrow $75,000 for a mortgage. Credit score is 720. Annual income is $150k."
if client:
    run_bank_orchestration = run_banking_squad(raw_app)

✅ Gemini client configured for **gemini-3-pro-preview**.
🚀 Starting Agentic Handoff...
Step 1: Processor Agent (Thinking: Low) is extracting data...
Step 2: Underwriter Agent (Thinking: High) is deciding...

📄 FINAL BANKING DECISION:
```json
{
  "decision": "FLAG for Manual Review",
  "reasoning": "While the credit score (720) meets the approval criteria (>700), the loan amount of $75,000 exceeds the $50,000 threshold for automatic approval.",
  "risk_score": 20
}
```


In [6]:
import google.genai as genai
from google.genai import types
import os
import json
from warnings import filterwarnings
filterwarnings('ignore')

# --- 1. SETUP ---
GEMINI_API_KEY = None
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

REQUESTED_MODEL_ID = 'gemini-3-pro-preview'
client = genai.Client(api_key=GEMINI_API_KEY) if GEMINI_API_KEY else None

# --- 2. AGENT DEFINITIONS ---

def agent_processor(raw_text):
    """AGENT 1: Data Extraction (Low Thinking)"""
    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_level="low"),
        temperature=0.1
    )
    # The Processor extracts the numerical data needed for the rest of the squad.
    prompt = f"Extract into JSON: loan_amount, credit_score, annual_income. Text: {raw_text}"
    response = client.models.generate_content(model=REQUESTED_MODEL_ID, config=config, contents=prompt)
    return response.text.replace('```json', '').replace('```', '').strip()

def agent_fraud_specialist(extracted_data):
    """AGENT 2: Risk & Fraud Analysis (Medium Thinking)"""
    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_level="medium"),
        temperature=0.5
    )
    # This agent checks for specific red flags like high debt-to-income ratios.
    prompt = f"""
    Analyze these financials for anomalies: {extracted_data}
    RULE: Flag RED if loan_amount is > 50% of annual_income. Otherwise GREEN.
    Return JSON ONLY: {{"fraud_status": "RED/GREEN", "risk_note": "reasoning"}}
    """
    response = client.models.generate_content(model=REQUESTED_MODEL_ID, config=config, contents=prompt)
    return response.text.replace('```json', '').replace('```', '').strip()

def agent_senior_underwriter(data_summary, fraud_report):
    """AGENT 3: Final Decision Logic (High Thinking)"""
    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_level="high", include_thoughts=True),
        temperature=1.0
    )
    # The Underwriter applies complex multi-step policy logic to reach a final status.
    prompt = f"""
    Policy:
    - REJECT if Credit < 600.
    - AUTO_APPROVE if Credit > 780 AND Loan < $100k AND Fraud is GREEN.
    - FLAGGED if Loan > $50k OR Fraud is RED.

    Data: {data_summary}
    Fraud Report: {fraud_report}

    Output JSON: {{"decision": "AUTO_APPROVE/AUTO_REJECT/FLAGGED", "reasoning": "string", "risk_score": 1-100}}
    """
    return client.models.generate_content(model=REQUESTED_MODEL_ID, config=config, contents=prompt)

# --- 3. HUMAN GATEKEEPER ---

def human_approval_gate(ai_json):
    """Halts the code and waits for user input if the AI flags a loan."""
    data = json.loads(ai_json.replace('```json', '').replace('```', '').strip())
    if data.get("decision") == "FLAGGED":
        print(f"\n⚠️  [HUMAN REVIEW] Reason: {data.get('reasoning')}")
        choice = input("Authorize exception? (yes/no): ").lower()
        return "APPROVED_BY_HUMAN" if choice == 'yes' else "REJECTED_BY_HUMAN"
    return data.get("decision")

# --- 4. ORCHESTRATION ---

def run_banking_squad(user_query):
    if not client: return "API Client not found."

    print("🚀 Step 1: Extracting Data...")
    raw_data = agent_processor(user_query)

    print("🚀 Step 2: Running Fraud Check...")
    fraud_report = agent_fraud_specialist(raw_data)

    print("🚀 Step 3: Senior Underwriter Analysis...")
    final_response = agent_senior_underwriter(raw_data, fraud_report)

    # Access and print the Internal Audit (thoughts) from the high-thinking model.
    for part in final_response.candidates[0].content.parts:
        if part.thought:
            print(f"\n🧠 [INTERNAL AUDIT]:\n{part.text}")

    # Process the final outcome via the human gatekeeper.
    final_status = human_approval_gate(final_response.text)
    print(f"\n✅ FINAL SYSTEM STATUS: {final_status}")

# --- EXECUTION ---
application = "Alice wants $75,000. Credit score 720. Income $150,000."
run_banking_squad(application)

🚀 Step 1: Extracting Data...
🚀 Step 2: Running Fraud Check...
🚀 Step 3: Senior Underwriter Analysis...

🧠 [INTERNAL AUDIT]:
**Application Decision Analysis and Output Generation**

Okay, so I've got this application in front of me, and I need to make a decision based on these specific policies. The first thing I always do is break down the requirements. It's like a logical puzzle.

I'm starting by checking if the application should be outright rejected. The policy says *reject* if the credit score is below 600. The applicant's credit is 720, so that's a no-go on rejection. Good.

Next, I look for the auto-approve criteria: credit above 780, loan under $100k, and a "GREEN" fraud status. The loan *is* under $100k, and the fraud is "GREEN," but the credit score of 720 falls short of that 780 threshold. So, no auto-approval here.

Now, I check for the flagged conditions.  It's an "OR" statement, meaning if *either* the loan exceeds $50k *or* the fraud is "RED," the application gets flagged